In [1]:
import itertools
import random

import pandas as pd
from sklearn import linear_model

from splitgraph import Repository, ResultShape
from splitgraph.ingestion.pandas import df_to_table, sql_to_df

PATH = "./iris.csv"

In [2]:
def load_dataset():
    """Loads the Iris dataset from CSV"""
    data = pd.read_csv(PATH,
                      names=['sepal_length', 'sepal_width',
                             'petal_length', 'petal_width', 'class'])
    data.index.names = ["id"]
    return data

iris = load_dataset()
iris

,sepal_length,sepal_width,petal_length,petal_width,class
id,,,,,
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica


In [3]:
# Initialize an empty Splitgraph repository and insert data into a staging area.

dataset = Repository("uciml", "iris")
dataset.delete()
dataset.objects.cleanup()
dataset.init()

df_to_table(iris, repository=dataset, table="data")

In [4]:
# Commit the repository as a new image and give it a tag.

base_image = dataset.commit()
base_image.tag("base")

print(base_image)
print(base_image.get_tables())

2019-09-10 14:44:33,882 [15214] INFO Committing uciml/iris...
2019-09-10 14:44:33,921 [15214] INFO Using temporary table o1af36270e197e18c0f80db369721dc77e69bb2f310c87604fe59d1bc50f263 for uciml/iris/data limit 10000 after_pk None


Image(image_hash='703b3969c678ae51994d8e01897f7ef7ccb8b23bf199094a129a9fbbbb6e03a0', parent_id='0000000000000000000000000000000000000000000000000000000000000000', created=datetime.datetime(2019, 9, 10, 14, 44, 33, 908357), comment=None, provenance_type=None, provenance_data=None, repository=Repository uciml/iris on LOCAL (object engine LOCAL), engine=PostgresEngine LOCAL (sgr@localhost:5432/splitgraph), object_engine=PostgresEngine LOCAL (sgr@localhost:5432/splitgraph))
['data']


In [5]:
# Generate a training/validation dataset by splitting the dataset in two

all_ids = dataset.run_sql("SELECT id FROM data",
                          return_shape=ResultShape.MANY_ONE)
validation_ids = random.sample(all_ids, len(all_ids) // 2)

dataset.run_sql("DELETE FROM data WHERE id IN (" 
                + ",".join(itertools.repeat("%s", len(validation_ids))) + ")",
               validation_ids)
training_image = dataset.commit()
training_image.tag("training")

2019-09-10 14:44:34,142 [15214] INFO Committing uciml/iris...


In [6]:
base_image.checkout()
dataset.run_sql("DELETE FROM data WHERE id NOT IN (" 
                + ",".join(itertools.repeat("%s", len(validation_ids))) + ")",
               validation_ids)
validation_image = dataset.commit()
validation_image.tag("validation")

2019-09-10 14:44:34,410 [15214] INFO Using cached objects list
2019-09-10 14:44:34,412 [15214] INFO Claiming 1 object(s)
2019-09-10 14:44:34,420 [15214] INFO Yielding to the caller
2019-09-10 14:44:34,484 [15214] INFO Applying 1 fragment(s)...
2019-09-10 14:44:34,502 [15214] INFO Releasing 1 object(s)
2019-09-10 14:44:34,504 [15214] INFO Timing stats for uciml/iris/703b3969c678ae51994d8e01897f7ef7ccb8b23bf199094a129a9fbbbb6e03a0/data: 
claim_objects: 0.008
prepare_fetch_list: 0.001
fetch_objects: 0.000
caller: 0.078
release_objects: 0.004
Total: 0.092
2019-09-10 14:44:34,549 [15214] INFO Committing uciml/iris...


In [7]:
def load_dataset_from_sg(image):
    # This uses layered querying, which is a way to run read-only queries on
    # Splitgraph images without checking them out.
    data = sql_to_df("SELECT sepal_length, sepal_width, class FROM data",
                     image=image, use_lq=True)
    X = data.values.T[:2].T
    Y = data.values.T[2].T
    
    return X, Y


def train_model(image):
    # Train model on the data in a Splitgraph image
    X, Y = load_dataset_from_sg(image)
    model = linear_model.LogisticRegression(C=1e5, solver="lbfgs", multi_class="auto")
    model.fit(X, Y)
    return model


def test_model(model, image):
    # Test model on the data in a Splitgraph image
    X, Y = load_dataset_from_sg(image)
    return model.score(X, Y)

In [8]:
# We can now easily switch between different datasets to train and validate our model.
# In this case, we train it on half of the data and then check its score
# on the other half.

model = train_model(dataset.images["training"])

val_score = test_model(model, dataset.images["validation"])
train_score = test_model(model, dataset.images["training"])
full_score = test_model(model, dataset.images["base"])

print("======= Scores: =======")
print("Training dataset: %.2f\nTest dataset: %.2f\nFull dataset: %.2f" \
     % (train_score, val_score, full_score))


2019-09-10 14:44:35,271 [15214] INFO Mounting uciml/iris:24ec6137495fb686568dfa1e0fe52976fa0b44360fa8b8d5d358a082f78d6b71/data into o042f4630863d8afcf239e7bd6d9be118
2019-09-10 14:44:36,038 [15214] INFO Mounting uciml/iris:3589185050e1ecdb9eac2ba952b474689faa6fadd4f4ec4719006339708417d5/data into o6d82d5b77888106d67abd9167737a1e3
2019-09-10 14:44:36,743 [15214] INFO Mounting uciml/iris:24ec6137495fb686568dfa1e0fe52976fa0b44360fa8b8d5d358a082f78d6b71/data into o032dfc13f2d87260f66e95200361556f
2019-09-10 14:44:37,448 [15214] INFO Mounting uciml/iris:703b3969c678ae51994d8e01897f7ef7ccb8b23bf199094a129a9fbbbb6e03a0/data into o09959c045c594280d7c1aa697a004eeb


======= Scores: =======
Training dataset: 0.88
Test dataset: 0.76
Full dataset: 0.82
